In [8]:
import os
import numpy as np
from skimage.draw import polygon
import cv2
import json
from shutil import copyfile
from pathlib import Path
from skimage.measure import find_contours
import glob
import base64
import skimage


In [9]:
def getFileNameInfor(filepath):
    file_name = filepath.split('/')[-1]
    file_base = Path(filepath).stem
    fdir = os.path.dirname(os.path.abspath(filepath))
    return (fdir, file_name, file_base)

In [10]:
def boundary2Mask(xs, ys, img):
    mask = np.zeros(img.shape[0:2])
    rr, cc = polygon(xs, ys, img.shape)
    mask[rr, cc] = 255
    return mask


In [12]:
def mask2Boundary(mask_file, sample_ratio=5):
    mask = cv2.imread(mask_file)[:,:,0] 
    contours = find_contours(mask, 200)
    boundaries=[]
    for i, verts in enumerate(contours):
        new_vects = []
        for k, p in enumerate(verts):
            # Subtract the padding and flip (y, x) to (x, y)z
            if k % sample_ratio==0:
                new_vects.append(list(p))
        boundaries.append(new_vects)
    return boundaries

#masks = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/proj_outdir/1456/masks/0_yin.jpg'
#print(mask2Boundary(masks))

In [14]:
def masks2json(mask_file_ls, img_file, out_dir):
    img_dir, img_name, img_id = getFileNameInfor(img_file)
    json_file="{}/{}.json".format(out_dir,image_id)
    # image = skimage.io.imread(img_file)
    image = cv2.imread(img_file) 
    imageHeight = image.shape[0]    # 获得长宽
    imageWidth = image.shape[1]
    with open(img_file, 'rb') as f:   # 编码图片 base64
        imageData = base64.b64encode(f.read())
        imageData = str(imageData, encoding='utf-8')
    json_data = {
        'version': "4.5.6",
        "flags": {},
        'shapes': [],
        'imagePath': img_file,
        'imageData': imageData,
        'imageHeight': imageHeight,
        'imageWidth': imageWidth
    }
    
    for i,m in enumerate(mask_file_ls):
        m_dir, m_name, m_id = getFileNameInfor(m)
        className = m_id.split("_")[-1]
        boundary = mask2Boundary(m)[0]
        obj = {
            'ID': i,
            'label': className,
            'points': [],
            'group_id': None,
            'shape_type': "polygon",
            'flags': {},
        }
        for point in boundary:
            obj["points"].append([float(point[0]), float(point[1])])
        json_data['shapes'].append(obj)
    json_str = json.dumps(json_data, indent=4)
    with open('{}/{}.json'.format(out_dir, img_id), 'w') as json_file:
        json_file.write(json_str)
    copyfile(img_file, '{}/{}.png'.format(out_dir, img_id))

# test
# img_file = "../../samples/hpv_models/test/data/proj_outdir/1456/1456.png"
# img_dir, img_name, img_id = getFileNameInfor(img_file) 
# mask_dir= "{}/masks/".format(img_dir)
# mask_file_ls = glob.glob('{}/*.jpg'.format(mask_dir))
# out_dir = "/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/out_json"
# masks2json(mask_file_ls, img_file, out_dir)

In [16]:
import sys
ROOT_DIR = os.path.abspath("../../../")  # 指定根目录
sys.path.append(ROOT_DIR)  # 查找库的本地版本
from mrcnn import utils

def get_iou(roi1, roi2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    bb1={"y1":roi1[0], 'x1':roi1[1],"y2":roi1[2], 'x2':roi1[3]}
    bb2={"y1":roi2[0], 'x1':roi2[1],"y2":roi2[2], 'x2':roi2[3]}
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

def removeDuplicateObject(r, min_ratio=0.7):
    classid_ls = list(map(int, r['class_ids']))
    scores = r["scores"]
    masks = r["masks"]
    rois =r['rois']

    new_classid_ls = []
    new_scores = []
    new_masks = None
    removed_indexes = []
    for i in range(len(classid_ls)-1):
        for k in range(i+1, len(classid_ls)):
            if k in removed_indexes:
                continue
            mask_a = np.reshape(masks[:,:,i],(masks.shape[0],masks.shape[1],1))
            roi_a = rois[i]
            class_a = classid_ls[i]
            mask_b = np.reshape(masks[:,:,k],(masks.shape[0],masks.shape[1],1))
            roi_b = rois[k]
            class_b = classid_ls[k]
            #print(utils.compute_overlaps_masks(mask_a, mask_b))
            #print('as')
            if get_iou(roi_a,roi_b)>0.8:
            #if utils.compute_overlaps_masks(mask_a, mask_b)[0]>min_ratio:
                if class_a<class_b:
                    removed_indexes.append(k)
                else:
                    removed_indexes.append(i)
                continue
        if i in removed_indexes:
            continue
        else:
            new_classid_ls.append(classid_ls[i])
            new_scores.append(scores[i])
            if new_masks is None:
                new_masks=np.reshape(masks[:,:, i],(masks.shape[0],masks.shape[1],1))
            else:
                mask=np.reshape(masks[:,:, i],(masks.shape[0],masks.shape[1],1))
                new_masks=np.concatenate([new_masks, mask], axis=-1)
    return {'class_ids':new_classid_ls, "scores":new_scores, "masks":new_masks}

/home/hqyone/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hqyone/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hqyone/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hqyone/anaconda3/envs/tf1/lib/python3.7/site-pack

In [17]:
def predictToJson(image_path, model, out_dir):
    img_dir, img_name, img_id = getFileNameInfor(image_path)
    image = skimage.io.imread(image_path)
    imageHeight = image.shape[0]    # 获得长宽
    imageWidth = image.shape[1]
    rr = model.detect([image], verbose=1)[0]
    r = removeDuplicateObject(rr,0.6)
    with open(image_path, 'rb') as f:   # 编码图片 base64
        imageData = base64.b64encode(f.read())
        imageData = str(imageData, encoding='utf-8')
    json_data = {
        'version': "4.5.6",
        "flags": {},
        'shapes': [],
        'imagePath': img_dir,
        'imageData': imageData,
        'imageHeight': imageHeight,
        'imageWidth': imageWidth
    }
    for i in range(0, len(r['class_ids'])):
        class_id = int(r['class_ids'][i])
        className = ['BG', 'yin', 'yin-yang', 'yang']   # 把数字对应上标签
        score = int(r["scores"][i])
        mask = r["masks"][:, :, i]
        boundary = mask2Boundary(mask)
        obj = {
            'ID': i,
            'label': className[class_id],
            'points': [],
            'group_id': None,
            'shape_type': "polygon",
            'flags': {},
        }
        i = 0
        for point in boundary:
            obj["points"].append([float(point[0]), float(point[1])])
        json_data['shapes'].append(obj)
    json_str = json.dumps(json_data, indent=4)
    with open('{}/{}.json'.format(out_dir, img_id), 'w') as json_file:
        json_file.write(json_str)
    copyfile(image_path, '{}/{}.png'.format(out_dir, img_id))

In [66]:

from skimage import data
a = data.camera()
b = data.camera()*2
masks = np.concatenate([a, b], axis=-1)

print(masks[:,:,0])

k=np.array([1,2])
p=np.array([3,4])
#np.append(k, p)
#a[:,:,1]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [50]:
# The function is 
def json2masks(json_file, img_file, out_dir):
    img_dir, img_name, image_id = getFileNameInfor(img_file)
    img = cv2.imread(img_file) 
    with open(json_file) as JSON:
        data = json.load(JSON)
        image_name = data['imagePath']
        image_height = data['imageHeight']
        image_width = data['imageWidth']
        shapes = data["shapes"]
        if len(shapes)>0:
            mod_image_id=image_id.replace(" ","").replace("(","_").replace(")","")
            os.makedirs('{}/{}'.format(out_dir,mod_image_id), exist_ok=True)
            os.makedirs('{}/{}/masks'.format(out_dir,mod_image_id), exist_ok=True)
            copyfile(img_file, '{}/{}/{}.png'.format(out_dir,mod_image_id,mod_image_id))
            for i, s in enumerate(shapes):
                label = s['label']
                points = s['points']
                xs = []
                ys = []
                for p in points:
                    ys.append(p[0])
                    xs.append(p[1])
                if len(xs)>0 and len(ys)>0:
                    mask = boundary2Mask(xs, ys, img)
                    cv2.imwrite("{}/{}/masks/{}_{}.png".format(out_dir, mod_image_id,i,label), mask)
           

In [51]:
def json2cell_infor(json_file):
    with open(json_file) as JSON:
        data = json.load(JSON)
        image_name = data['imagePath']
        shapes = data["shapes"]
        cell_dic = {}
        for i, s in enumerate(shapes):
            label = s['label']
            if label not in cell_dic:
                cell_dic[label]=0
            cell_dic[label]+=1
        return cell_dic
            
            
    

In [52]:
test_img = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/datasource/1456.png'
image_id = '1456'
test_json = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/datasource/1459.json'
out_dir = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/stage1_train'
# 
# json2masks(test_json,test_img, out_dir)
# print(json2cell_infor(test_json))
import os

pictures_dir ="/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500"

for folder, dirs, files in os.walk(pictures_dir):
    for file in files:
        if file.endswith('.png'):
            png_file=os.path.join(folder,file)
            json_file = os.path.join(folder,file.replace(".png",'.json'))
            if os.path.isfile(json_file):
                if not os.path.isdir(out_dir):
                    os.mkdir(out_dir)
                print(f'{png_file}:{json_file}')
                json2masks(json_file,png_file, out_dir)


/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1456.png:/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1456.json
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1458.png:/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1458.json
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1459.png:/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1459.json
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1461.png:/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1461.json
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1462.png:/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1462.json
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1467.png:/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/500/1467.json
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_model

[]
['1456']
['27_yin.jpg', '45_yin.jpg', '0_yin.jpg', '10_yin.jpg', '11_yin.jpg', '12_yin.jpg', '13_yin.jpg', '14_yin.jpg', '15_yin.jpg', '16_yin.jpg', '17_yin.jpg', '18_yin.jpg', '19_yin.jpg', '1_yin.jpg', '20_yin.jpg', '21_yin.jpg', '22_yin.jpg', '23_yin.jpg', '24_yin.jpg', '25_yin.jpg', '26_yin.jpg', '28_yin.jpg', '29_yin.jpg', '2_yin.jpg', '30_yin.jpg', '31_yin.jpg', '32_yin.jpg', '33_yin.jpg', '34_yin.jpg', '35_yin.jpg', '36_yin.jpg', '37_yin.jpg', '38_yin.jpg', '39_yin.jpg', '3_yin.jpg', '40_yin.jpg', '41_yin.jpg', '42_yin.jpg', '43_yin.jpg', '44_yin.jpg', '46_yin.jpg', '47_yin.jpg', '48_yin.jpg', '49_yin.jpg', '4_yin.jpg', '50_yin.jpg', '51_yin.jpg', '52_yin.jpg', '53_yin.jpg', '54_yin.jpg', '55_yin.jpg', '56_yin.jpg', '57_yin.jpg', '58_yin.jpg', '59_yin.jpg', '5_yin.jpg', '60_yin.jpg', '61_yin.jpg', '62_yin.jpg', '63_yin.jpg', '64_yin.jpg', '65_yin.jpg', '66_yin.jpg', '67_yin.jpg', '68_yin.jpg', '69_yin.jpg', '6_yin.jpg', '70_yin.jpg', '71_yin.jpg', '72_yin.jpg', '7_yin.jpg', '

In [59]:
infor_path = "/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/stage1_train/xx_best_696_39_33/xx_best_696_39_33.png"
print(os.path.dirname(infor_path))
print(os.path.dirname(infor_path),"/masks")
print(os.path.join(os.path.dirname(infor_path),"masks"))
print(os.path.join(os.path.dirname(os.path.dirname(infor_path)),"masks"))

/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/stage1_train/xx_best_696_39_33
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/stage1_train/xx_best_696_39_33 /masks
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/stage1_train/xx_best_696_39_33/masks
/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/data/stage1_train/masks


In [1]:
import torch

ModuleNotFoundError: No module named 'torch'